In [1]:
import numpy as np
import tensorflow as tf

In [6]:
import sys
sys.path.append('..')
from OneD_RNN import OneD_RNN_wavefxn,RNNWavefunction1D


In [2]:
def optimizer_initializer(optimizer):
    fake_var = tf.Variable(1.0)
    with tf.GradientTape() as tape:
        fake_loss = tf.reduce_sum(fake_var ** 2)
    grads = tape.gradient(fake_loss, [fake_var])
    # Ask the optimizer to apply the processed gradients.
    optimizer.apply_gradients(zip(grads, [fake_var]))

In [13]:
 # ---- System Parameters -----------------------------------------------------------------
Lx = 16
Ly = 16
V = 9
delta = -1.545
Omega = 4.24
sweep_rate = 15

# ---- RNN Parameters ---------------------------------------------------------------------
num_hidden = 32
learning_rate = 1e-3
weight_sharing = True
trunc = 100
seed = 123
rnn_type = 'OneD'

# ---- Training Parameters ----------------------------------------------------------------
ns = 100
batch_samples = False
if batch_samples:
    batch_size_samples = config.get('batch_size_samples', 100)
    print(f"Batching samples drawn from RNN with batch size = {batch_size_samples}")
else:
    batch_size_samples = ns
    print(f"Not batching samples drawn from RNN, meaning batch size = {ns}")
batch_size_data = 100
data_epochs = 1000
vmc_epochs = 1000
total_epochs = data_epochs+vmc_epochs
# data = data_given_param(sweep_rate,delta)
# tf_dataset = create_KZ_tf_dataset(data)   
global_step = tf.Variable(0, name="global_step") 

Not batching samples drawn from RNN, meaning batch size = 100


In [14]:
wavefxn = OneD_RNN_wavefxn(Lx,Ly,num_hidden,learning_rate,seed)

OneD RNN used in previous Rydberg project.
gru_2/gru_cell_2/kernel:0 (192,)
gru_2/gru_cell_2/recurrent_kernel:0 (3072,)
gru_2/gru_cell_2/bias:0 (192,)
dense_1/kernel:0 (64,)
dense_1/bias:0 (2,)
The sum of params is 3522


In [15]:
exp_name = 'Test_new_checkpointing_1d'
path = f'./data/N_{Lx*Ly}/{exp_name}/{rnn_type}_rnn/delta_{delta}'
ckpt = tf.train.Checkpoint(step=global_step, optimizer=wavefxn.optimizer, variables=wavefxn.trainable_variables) # these are trackable objects - we get to specify
manager = tf.train.CheckpointManager(ckpt, path, max_to_keep=1) # change 1 to None

In [16]:
ckpt.restore(manager.latest_checkpoint)
latest_ckpt = ckpt.step.numpy()
print(latest_ckpt)

1000


In [22]:
ckpt.restore(manager.checkpoints[34])
latest_ckpt = ckpt.step.numpy()
print(latest_ckpt)

350


In [23]:
375%10

5

In [26]:
step = 344
index = int((step - (step%10))/10 - 1)

In [27]:
print(index)

33


In [28]:
restart_point = 113
index = int((restart_point-(restart_point%10))/10 -1)

In [29]:
print(index)

10


In [41]:
restart_point = 956
index = int((restart_point-(restart_point%10))/10 - 1)
if index < len(manager.checkpoints):
    ckpt.restore(manager.checkpoints[index])
    ckpt_found = True
else:
    manager.latest_checkpoint
    ckpt_found = False
if ckpt_found:
    print(f"CKPT ON and ckpt {index} found.")
    print("Restored from {}".format(manager.checkpoints[index]))
    ckpt_step = ckpt.step.numpy()
    optimizer_initializer(wavefxn.optimizer)
    print(f"Continuing at step {ckpt.step.numpy()}")
    energy = np.load(path+'/Energy.npy').tolist()[0:ckpt_step]
    variance = np.load(path+'/Variance.npy').tolist()[0:ckpt_step]
    cost = np.load(path+'/Cost.npy').tolist()[0:ckpt_step]
elif manager.latest_checkpoint:
    ckpt.restore(manager.latest_checkpoint) 
    print(f"CKPT ON but ckpt {index} not found.")
    print("Restored from {}".format(manager.latest_checkpoint))
    latest_ckpt = ckpt.step.numpy()
    optimizer_initializer(wavefxn.optimizer)
    print(f"Continuing at step {ckpt.step.numpy()}")
    energy = np.load(path+'/Energy.npy').tolist()[0:latest_ckpt]
    variance = np.load(path+'/Variance.npy').tolist()[0:latest_ckpt]
    cost = np.load(path+'/Cost.npy').tolist()[0:latest_ckpt]
else:
    print("CKPT ON but no ckpt found. Initializing from scratch.")
    latest_ckpt = 0
    energy = []
    variance = []
    cost = []

CKPT ON and ckpt 94 found.
Restored from ./data/N_256/Test_new_checkpointing_1d/OneD_rnn/delta_-1.545/ckpt-95
Continuing at step 950
950


In [35]:
print(manager.checkpoints[index])

./data/N_256/Test_new_checkpointing_1d/OneD_rnn/delta_-1.545/ckpt-97
